In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

2024-06-05 21:45:59.573127: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-05 21:45:59.841017: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 21:46:00.511778: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Applying the temporal fusion transformers gatting mechanismis using tensorflow 

The teoric text is in my final course work "", in 1.6.1 cap

In [2]:
class GLU(tf.keras.layers.Layer):
    def __init__(self,units):
        """Define a Gate Linear Unit

        Args:
            units (int): the units quantities in the dense layer on GLU
        """
        super(GLU,self).__init__()

        self.linear = tf.keras.layers.Dense(units)                      #creating a layer to calculate W_{5}x+b_{5}
        self.dense_sigmoid = tf.keras.Dense(units,activation='sigmoid') #creating a layer to calculate sigmoid(W_{4}x+b_{4})

        def call(self,inputs):
            """Calculate the element wise multiplication between the layers

            Args:
                inputs (tf.Tensor,tf.Dataset or np.array): the input to calculate the GLU

            Returns:
                tf.Tensor,tf.Dataset or np.array: element_wise_product
            """
            element_wise_product = self.dense_sigmoid(inputs) * self.linear(inputs)
            return element_wise_product

In [3]:
class Drop_GLU_Add_Norm(tf.keras.layers.Layer):
    def __init__(self, units, drop_rate):
        """Define the process of apply a dropout, a GLU add the residuals to GLU output and normalize them

        Args:
            units (int): the units quantities in the dense layer on GLU
            drop_rate (float): a float number 0 <= drop_rate <= 1, where define te dropout rate
        """
        super(Drop_GLU_Add_Norm,self).__init__()
        self.units= units
        self.drop_rate = drop_rate

        self.dropout_layer = tf.keras.layers.Dropout(self.drop_rate)
        self.layer_GLU = GLU(self.units)
        self.norm_layer = tf.keras.layers.LayerNormalization()

    def call(self,inputs, residual):
        """Compute all process 

        Args:
            inputs (tf.Tensor,tf.Dataset or np.array): real input
            residual (tf.Tensor,tf.Dataset or np.array): the residuals input to add to normalize
        Returns:
            tf.Tensor,tf.Dataset or np.array : normalized_values
        """
        input_droped = self.dropout_layer(inputs)
        glu_output = self.layer_GLU(input_droped)
        normalized_values = self.norm_layer(glu_output + residual)
        return normalized_values

In [4]:
class GRN(tf.keras.layers.Layer):
    def __init__(self, units, drop_rate, optional_context=False):
        super(GRN,self).__init__()

        self.units = units
        self.drop_rate = drop_rate
        self.optional_context = optional_context

        self.layer_ELU = tf.keras.layers.ELU()
        self.first_linear = tf.keras.layers.Dense(self.units)
        self.second_linear = tf.keras.layers.Dense(self.units)

        if self.optional_context:
            self.linear_optioal = tf.keras.layers.Dense(self.units,use_bias=False)

        
        self.add_norm = Drop_GLU_Add_Norm(units=self.units,
                                          drop_rate=self.drop_rate)
        
    def call(self,inputs):
        if self.optional_context:
            X, context = inputs
            dense_out = self.first_linear(X)
            context_out = self.linear_optioal(context)
            first_output = self.layer_ELU(dense_out + context_out)
        else:
            X = inputs
            dense_out = self.first_linear(X)
            first_output = self.layer_ELU(dense_out)

        second_output = self.second_linear(first_output)

        final_output = self.add_norm(second_output,X)

        return final_output
